In [1]:
import pandas as pd

metadata = pd.read_csv('metadata_mod.csv', low_memory=False)

metadata.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,2,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,...,21-10-1988,0,69,"[{'iso_639_1': 'fi', 'name': 'suomi'}, {'iso_6...",Released,NaN,Ariel,False,7.1,44
1,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,3,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",...,16-10-1986,0,76,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Shadows in Paradise,False,7.1,35
2,False,NaN,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,5,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,...,09-12-1995,4300000,98,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6.5,539
3,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,6,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",...,15-10-1993,12136938,110,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.4,79
4,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.starwars.com/films/star-wars-episod...,11,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,...,25-05-1977,775398007,121,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8.1,6778


In [2]:
C = metadata['vote_average'].mean()
print(C)

7.002102102102103


In [3]:
m = metadata['vote_count'].quantile(0.90)
print(m)

2841.0


In [4]:
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(101, 24)

In [5]:
metadata.shape

(999, 24)

In [6]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [7]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)


In [8]:
q_movies = q_movies.sort_values('score', ascending=False)

q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

,title,vote_count,vote_average,score
223,The Shawshank Redemption,8358,8.5,8.120008
112,The Dark Knight,12269,8.3,8.055968
186,The Godfather,6024,8.5,8.019963
414,Fight Club,9678,8.3,8.005461
535,Pulp Fiction,8670,8.3,7.979669
6,Forrest Gump,8147,8.2,7.890278
84,The Lord of the Rings: The Return of the King,8226,8.1,7.818160
322,Schindler's List,4436,8.3,7.793290
4,Star Wars,6778,8.1,7.775733
89,Spirited Away,3968,8.3,7.758463


In [9]:
metadata['overview'].head()


0    Taisto Kasurinen is a Finnish coal miner whose...
1    An episode in the life of Nikander, a garbage ...
2    It's Ted the Bellhop's first night on the job....
3    While racing to a boxing match, Frank, Mike, J...
4    Princess Leia is captured and held hostage by ...
Name: overview, dtype: object

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
metadata['overview'] = metadata['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(metadata['overview'])
tfidf_matrix.shape

(999, 9122)

In [11]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [12]:
cosine_sim.shape

(999, 999)

In [13]:
cosine_sim[1]

array([0.01690673, 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.0501316 , 0.04014904, 0.01646768, 0.02596633,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.03531255, 0.        , 0.        , 0.03761027,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01745644, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.04991632,
       0.01558686, 0.01548789, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01588039, 0.03237902,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01025347, 0.        , 0.        , 0.01539372, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.04980729, 0.03775695, 0.        , 0.01685288,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.11578349, 0.        , 0.        , 0.     

In [14]:
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()


In [15]:
indices[:10]

title
Ariel                  0
Shadows in Paradise    1
Four Rooms             2
Judgment Night         3
Star Wars              4
Finding Nemo           5
Forrest Gump           6
American Beauty        7
Citizen Kane           8
Dancer in the Dark     9
dtype: int64

In [16]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    return metadata['title'].iloc[movie_indices]


In [17]:
get_recommendations('The Terminator')

225            Terminator 2: Judgment Day
401                  Terminator Salvation
238    Terminator 3: Rise of the Machines
10                               The Dark
300                     Kill Bill: Vol. 2
711                          Delicatessen
462                            The Matrix
137                       Minority Report
606                                Gandhi
765                            Spaceballs
Name: title, dtype: object

In [18]:
credits = pd.read_csv('credit_mod.csv')
keywords = pd.read_csv('keyword_mod.csv')

keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

C:\Users\SURESH CHOUDHARY H\AppData\Local\Temp\ipykernel_4952\732359111.py:1: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253

In [19]:
metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,Unnamed: 1128,Unnamed: 1129,Unnamed: 1130,Unnamed: 1131,Unnamed: 1132,Unnamed: 1133,Unnamed: 1134,Unnamed: 1135,Unnamed: 1136,keywords
0,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,2,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 240, 'name': 'underdog'}, {'id': 378, ..."
1,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,3,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 1361, 'name': 'salesclerk'}, {'id': 17..."


In [20]:
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)


In [21]:
import numpy as np


In [22]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan


In [23]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names

    return []


In [24]:
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)


In [25]:
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)


,title,cast,director,keywords,genres
0,Ariel,"[Turo Pajala, Susanna Haavisto, Matti PellonpÃ...",Aki KaurismÃ¤ki,"[underdog, prison, factory worker]","[Drama, Crime]"
1,Shadows in Paradise,"[Matti PellonpÃ¤Ã¤, Kati Outinen, Sakari Kuosm...",Aki KaurismÃ¤ki,"[salesclerk, helsinki, garbage]","[Drama, Comedy]"
2,Four Rooms,"[Tim Roth, Antonio Banderas, Jennifer Beals]",Allison Anders,"[hotel, new year's eve, witch]","[Crime, Comedy]"


In [26]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


In [27]:
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)


In [28]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])


In [29]:
metadata['soup'] = metadata.apply(create_soup, axis=1)


In [30]:
metadata[['soup']].head(2)


,soup
0,underdog prison factoryworker turopajala susan...
1,salesclerk helsinki garbage mattipellonpã¤ã¤ k...


In [31]:
metadata[['soup'][0]]

0      underdog prison factoryworker turopajala susan...
1      salesclerk helsinki garbage mattipellonpã¤ã¤ k...
2      hotel newyear'seve witch timroth antoniobander...
3      chicago drugdealer boxingmatch emilioestevez c...
4      android galaxy hermit markhamill harrisonford ...
                             ...                        
991    cheerleader sport highschool kirstendunst jess...
992    musician music ireland colmmeaney bernardhill ...
993    beethoven womandirector edharris dianekruger m...
994    lossoffamily emigration worldwarii julianekã¶h...
995    corruption bishop courtcase richardgere edward...
Name: soup, Length: 996, dtype: object

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = (count.fit_transform(metadata['soup'])).astype(np.int8)


In [33]:
count_matrix.shape


(996, 3716)

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)


In [35]:
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title'])

In [36]:
print(type(cosine_sim))

<class 'numpy.ndarray'>


In [37]:
print(cosine_sim2[0].shape)

(996,)


In [39]:
get_recommendations('Aliens', cosine_sim2)


283                         Alien
167                The Terminator
126                    Predator 2
424             Starship Troopers
549                     Moonraker
576           From Dusk Till Dawn
686            Planet of the Apes
225    Terminator 2: Judgment Day
611                       The Fog
423                      Die Hard
Name: title, dtype: object